1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()
connection_string = 'mysql+pymysql://root:' + password + '@localhost/Sakila'
engine = create_engine(connection_string)

········


In [8]:
data_film = pd.read_sql_table("film", engine)
display(data_film.head())
data_inventory = pd.read_sql_table("inventory", engine)
display(data_inventory.head())
data_rental = pd.read_sql_table("rental", engine)
display(data_rental.head())

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"{Commentaries, Behind the Scenes}",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,{Deleted Scenes},2006-02-15 05:03:42


/Users/faryal/anaconda3/lib/python3.11/site-packages/pandas/io/sql.py:1770: SAWarning: Did not recognize type 'geometry' of column 'location'
  tbl = Table(


,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


/Users/faryal/anaconda3/lib/python3.11/site-packages/pandas/io/sql.py:1770: SAWarning: Did not recognize type 'geometry' of column 'location'
  tbl = Table(


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.
3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

In [44]:
query = '''
    SELECT
      DISTINCT title, release_year, length, rating, special_features, language_id,
      CASE
        WHEN EXISTS (
          SELECT 1
          FROM rental
          INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id
          WHERE inventory.film_id = film.film_id
          AND MONTH(rental_date) = 5
          AND YEAR(rental_date) = 2005
        ) THEN TRUE
        ELSE FALSE
      END AS rented_in_may
    FROM film;
'''

df = pd.read_sql_query(query, engine)
display(df.head(1000))
display(df.shape)

,title,release_year,length,rating,special_features,language_id,rented_in_may
0,ACADEMY DINOSAUR,2006,86,PG,"Deleted Scenes,Behind the Scenes",1,1
1,ACE GOLDFINGER,2006,48,G,"Trailers,Deleted Scenes",1,0
2,ADAPTATION HOLES,2006,50,NC-17,"Trailers,Deleted Scenes",1,1
3,AFFAIR PREJUDICE,2006,117,G,"Commentaries,Behind the Scenes",1,1
4,AFRICAN EGG,2006,130,G,Deleted Scenes,1,1
...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,2006,183,G,"Trailers,Behind the Scenes",1,0
996,YOUTH KICK,2006,179,NC-17,"Trailers,Behind the Scenes",1,0
997,ZHIVAGO CORE,2006,105,NC-17,Deleted Scenes,1,1
998,ZOOLANDER FICTION,2006,101,R,"Trailers,Deleted Scenes",1,1


(1000, 7)

4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Numerical Features
# release_year
# length

# Categorical Features
# rating
# special_features
# language_id

# Encode categorical variables
df['language_id'] = df['language_id'].astype('category').cat.codes
df['rating'] = df['rating'].astype('category').cat.codes
df['special_features'] = df['special_features'].astype('category').cat.codes

# Scale numerical variables
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['release_year', 'length']] = scaler.fit_transform(df[['release_year', 'length']])

display(df.head(1000))

# Split the data into training and testing sets
X = df[['language_id', 'rating', 'special_features', 'release_year', 'length']]
y = df['rented_in_may']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print('Accuracy:', accuracy)
score = model.score(X_test, y_test)
print('Model score:', score)

# Make predictions for all films in the dataset
predictions = model.predict_proba(X_test)

# Calculate the probability that a film will be rented in May 2005
probability_rented_in_may = predictions[:, 1]

# Add the probability that a film will be rented in May 2005 to the dataset
X_test['probability_rented_in_may'] = probability_rented_in_may

# Print the probability that a film will be rented in May 2005 for all films in the dataset
d = pd.DataFrame(X_test)
d
# Model is about 70% accurate, which is decent
# We can use the model now to predict the probability of the movies being rented out in may,
# or have a new movie in the database and get the probability of it being rented in may.

,title,release_year,length,rating,special_features,language_id,rented_in_may,probability_rented_in_may
0,ACADEMY DINOSAUR,0.0,-0.724445,2,6,0,1,0.682141
1,ACE GOLDFINGER,0.0,-1.664897,0,13,0,0,0.662220
2,ADAPTATION HOLES,0.0,-1.615399,1,13,0,1,0.666123
3,AFFAIR PREJUDICE,0.0,0.042766,0,2,0,1,0.682303
4,AFRICAN EGG,0.0,0.364499,0,5,0,1,0.678704
...,...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,0.0,1.676182,0,8,0,0,0.677371
996,YOUTH KICK,0.0,1.577187,1,8,0,0,0.680843
997,ZHIVAGO CORE,0.0,-0.254219,1,5,0,1,0.680977
998,ZOOLANDER FICTION,0.0,-0.353214,4,13,0,1,0.680257


Accuracy: 0.696
Model score: 0.696


,language_id,rating,special_features,release_year,length,probability_rented_in_may
249,0,1,6,0.0,1.354449,0.683217
353,0,0,12,0.0,-0.749194,0.665860
537,0,3,8,0.0,1.156459,0.687221
424,0,4,0,0.0,-0.674947,0.698044
564,0,3,4,0.0,-1.466907,0.686987
...,...,...,...,...,...,...
662,0,1,9,0.0,-0.402711,0.674827
454,0,1,5,0.0,-0.303716,0.680864
474,0,0,5,0.0,0.488243,0.678988
917,0,2,11,0.0,0.908971,0.678660
